In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-data-for-ml-course/train.csv
/kaggle/input/home-data-for-ml-course/train.csv.gz
/kaggle/input/home-data-for-ml-course/sample_submission.csv.gz
/kaggle/input/home-data-for-ml-course/test.csv.gz
/kaggle/input/home-data-for-ml-course/test.csv
/kaggle/input/home-data-for-ml-course/sample_submission.csv
/kaggle/input/home-data-for-ml-course/data_description.txt


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
X_full = pd.read_csv('../input/home-data-for-ml-course/train.csv', index_col='Id')
X_test = pd.read_csv('../input/home-data-for-ml-course/test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

# # To keep things simple, we'll use only numerical predictors
# X = X_full.select_dtypes(exclude=['object'])
# X_test = X_test_full.select_dtypes(exclude=['object'])

# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X_full, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [3]:
# dropping columns having more than 100 null values

col = [c for c in X_train.columns if X_train[c].isnull().sum()>100] #X_train[X_train.columns].isnull().sum()>100

red_X_train = X_train.drop(col, axis=1)
red_X_valid = X_valid.drop(col, axis=1)
red_X_test = X_test.drop(col, axis=1)

In [4]:
from sklearn.preprocessing import OneHotEncoder

obj = red_X_train.select_dtypes('object')
obj_col= obj.columns
#print(obj_col.shape) 
#red_X_train[obj_col].replace({np.nan:'0'}, inplace=True)
red_X_train[obj_col]=red_X_train[obj_col].fillna('unknwon')
red_X_valid[obj_col] = red_X_valid[obj_col].fillna('unknown')
red_X_test[obj_col] = red_X_test[obj_col].fillna('unknown')


onehot = OneHotEncoder(handle_unknown='ignore' , sparse = False)
oh_X_train = pd.DataFrame(onehot.fit_transform(red_X_train[obj_col]))
oh_X_valid = pd.DataFrame(onehot.transform(red_X_valid[obj_col]))
oh_X_test = pd.DataFrame(onehot.transform(red_X_test[obj_col]))


oh_X_train.index = red_X_train.index
oh_X_valid.index = red_X_valid.index
oh_X_test.index = red_X_test.index

#print(red_X_train.shape)
red_X_train = red_X_train.drop(obj_col, axis=1)
red_X_valid = red_X_valid.drop(obj_col, axis=1)
red_X_test = red_X_test.drop(obj_col, axis=1)

#print(red_X_train.shape, oh_X_train.shape)
oh_X_train = pd.concat([red_X_train, oh_X_train], axis=1)
oh_X_valid = pd.concat([red_X_valid, oh_X_valid], axis=1)
oh_X_test = pd.concat([red_X_test, oh_X_test], axis=1)

#print(oh_X_train.shape)

In [5]:
# imputing the remaining null values  

from sklearn.impute import SimpleImputer
 
x = SimpleImputer(strategy='mean')
X_train_imputed = pd.DataFrame(x.fit_transform( oh_X_train ))
X_valid_imputed = pd.DataFrame(x.transform( oh_X_valid ))
X_test_imputed = pd.DataFrame(x.transform( oh_X_test ))


X_train_imputed.columns = oh_X_train.columns
X_valid_imputed.columns = oh_X_valid.columns
X_test_imputed.columns = oh_X_test.columns



In [6]:
from xgboost import XGBRegressor

xgb = XGBRegressor(n_estimators=100000, learning_rate=0.01)
xgb.fit(X_train_imputed, y_train, early_stopping_rounds=20, eval_set=[(X_valid_imputed, y_valid)], verbose=False)

pred = xgb.predict(X_valid_imputed)
preds_test = xgb.predict(X_test_imputed)

print('mean_absolute_error:')



mean_absolute_error:


NameError: name 'mean_absolute_error' is not defined

In [ ]:
# Save test predictions to file
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)

In [ ]:
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import mean_absolute_error
# from sklearn.svm import SVC

# model=SVC()
# parameters={
#     'kernel': ['linear'],
#     'C': [0.1]
# }
# cv = GridSearchCV(model,parameters,cv=5)
# cv.fit(X_train_imputed, y_train.values.ravel())
# pred=cv.predict(X_valid_imputed)
# print(mean_absolute_error(y_valid,pred))